In [1]:
import pandas as pd
from swmm_api.input_file import read_inp_file
from swmm_api.input_file.section_labels import *
from swmm_api.input_file.sections import *
from swmm_api.input_file.helpers import *
from swmm_api.input_file.sections.others import TimeseriesData
from swmm_api.input_file.section_labels import TIMESERIES
import os
os.environ['CONDA_DLL_SEARCH_MODIFICATION_ENABLE']="1"
import numpy as np
import shutil
import yaml
from pyswmm import Simulation,Links,Nodes,RainGages,SystemStats
import matplotlib.pyplot as plt

config = yaml.load(open("./states_yaml/chaohu.yaml"), yaml.FullLoader)
import step_sim
import GI_setting as gs

params={
    'kf':1,
    'kc':1,
    'advance_seconds':30,
}

e:\anaconda3\envs\tensorflow-cpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# S1: RSH

In [2]:
raine = np.load('./RSH/east.npy').tolist()
rainw = np.load('./RSH/east.npy').tolist()

for rainid in range(6):
    inp1 = read_inp_file('./S12/chaohu_.inp')
    inp1[TIMESERIES]['rainfalle']=TimeseriesData('rainfalle',raine[rainid])
    inp1[TIMESERIES]['rainfallw']=TimeseriesData('rainfallw',rainw[rainid])
    inp1.write_file('./S12/Results/RSH/chaohu_rain_'+str(rainid)+'.inp')
    step_sim.simfile('./S12/Results/RSH/',rainid,config,params)

# S2: R

In [3]:
raine = np.load('./RealRain/real.npy').tolist()
rainw = np.load('./RealRain/real.npy').tolist()

for rainid in range(len(raine)):
    inp1 = read_inp_file('./S12/chaohu_.inp') 
    inp1[TIMESERIES]['rainfalle']=TimeseriesData('rainfalle',raine[rainid])
    inp1[TIMESERIES]['rainfallw']=TimeseriesData('rainfallw',rainw[rainid])
    inp1.write_file('./S12/Results/RR/chaohu_rain_'+str(rainid)+'.inp')
    step_sim.simfile('./S12/Results/RR/',rainid,config,params)

# S3: RSH-GR

In [ ]:
raine = np.load('./RSH/east.npy').tolist()
rainw = np.load('./RSH/east.npy').tolist()

for rainid in range(6):
    inp = read_inp_file('./S34_GR/chaohu_GI.inp')
    # GI
    #add_GI(inp)
    # rain
    inp[TIMESERIES]['rainfalle']=TimeseriesData('rainfalle',raine[rainid])
    inp[TIMESERIES]['rainfallw']=TimeseriesData('rainfallw',rainw[rainid])
    inp.write_file('./S34_GR/Results/RSH/chaohu_rain_'+str(rainid)+'.inp')
    step_sim.simfile('./S34_GR/Results/RSH/',rainid,config,params)

# S4: R-GR

In [ ]:
raine = np.load('./RealRain/real.npy').tolist()
rainw = np.load('./RealRain/real.npy').tolist()

for rainid in range(len(raine)):
    inp = read_inp_file('./S34_GR/chaohu_GI.inp')
    # GI
    #add_GI(inp)
    # rain
    inp[TIMESERIES]['rainfalle']=TimeseriesData('rainfalle',raine[rainid])
    inp[TIMESERIES]['rainfallw']=TimeseriesData('rainfallw',rainw[rainid])
    inp.write_file('./S34_GR/Results/RR/chaohu_rain_'+str(rainid)+'.inp')
    step_sim.simfile('./S34_GR/Results/RR/',rainid,config,params)